In [21]:
import json

file_path = 'result_llama-2-7b-chat_multiarith_decomp_llama-2-7b-chat_direct.json'

with open(file_path, 'r') as file:
    data = json.load(file)

for question in data[:1]:
    print(question['sub_questions_answers'][question['sub_questions_answers'].index("Question 5:"):])

Question 5:  Paige had 11 songs on her mp3 player. If she deleted 9 old songs from it and then added 8 new songs, how many songs does she have on her mp3 player? 
Question 5.1: How many songs did Paige delete from her mp3 player?
Answer 5.1: 9 old songs were deleted from her mp3 player.

Question 5.2: How many new songs did Paige add to her mp3 player?
Answer 5.2: 8 new songs were added to her mp3 player.

Question 5.3: Now we can answer the question: How many songs does Paige have on her mp3 player?
Answer 5.3: 11 - 9 = 2 songs were deleted. 8 new songs were added. So Paige has 2 + 8 = 10 songs on her mp3 player. The answer is 10.


In [20]:
file_path = 'result_llama-2-7b-chat_multiarith_decomp_llama-2-7b-chat_mathreg.json'

with open(file_path, 'r') as file:
    data = json.load(file)

for question in data[:1]:
    print(question['sub_questions_answers'][question['sub_questions_answers'].index("Question 5:"):])

Question 5:  Paige had 11 songs on her mp3 player. If she deleted 9 old songs from it and then added 8 new songs, how many songs does she have on her mp3 player? 
Question 5.1: How many songs did Paige delete from her mp3 player?
Answer 5.1: 9 old songs were deleted from her mp3 player.
Question 5.2: How many new songs did Paige add to her mp3 player?
Answer 5.2: 8 new songs were added to her mp3 player.
Question 5.3: Now we can answer the question: How many songs does Paige have on her mp3 player?
Answer 5.3: 11 - 9 = 2.000000 songs were deleted from her mp3 player. Then 8 new songs were added to her mp3 player. So Paige has 2 + 8 = 10.000000 songs on her mp3 player. The answer is 2
